<a href="https://colab.research.google.com/github/Tirthankar4/Encryptix/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fraud-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F817870%2F1399887%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240719%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240719T103725Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D02515603955e5ad55421caaff332390ace476da2aa0112993811b655625b036095a352dd4c808d2db76673b5eca0fece7faa0b185644b98cc70d448aa7f802d9f8ef9a676eabeb5168fe6c52fa5b5dd41878f32450f39289f3252c956287f566a61b5c6f11d3d4df79ab46afdc6e42b2878e5a39811e06840a6bf741bcaa05dbb85bc18ce2a3a5ed7ae83a60b0355067e0ed3c74ba992e28520ab017b8715c814b87d697a5685e3e551bfec4a9eb8379b7bc614fd23e5b5f461df92619e53dfab632c6a32d372c920c248adbe06be8974c8508e940469071354f8ea0cfde92620ad73045a743420efef972f5222c1aeca60dd48d707390dcc7d5de762c1975b1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import optuna
from sklearn.metrics import roc_auc_score, classification_report
import warnings

warnings.filterwarnings("ignore")

Import Dataset and get Basic Info

In [ ]:
X = pd.read_csv('/kaggle/input/fraud-detection/fraudTrain.csv', index_col = 'Unnamed: 0')
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1296675 entries, 0 to 1296674
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1296675 non-null  object 
 1   cc_num                 1296675 non-null  int64  
 2   merchant               1296675 non-null  object 
 3   category               1296675 non-null  object 
 4   amt                    1296675 non-null  float64
 5   first                  1296675 non-null  object 
 6   last                   1296675 non-null  object 
 7   gender                 1296675 non-null  object 
 8   street                 1296675 non-null  object 
 9   city                   1296675 non-null  object 
 10  state                  1296675 non-null  object 
 11  zip                    1296675 non-null  int64  
 12  lat                    1296675 non-null  float64
 13  long                   1296675 non-null  float64
 14  city_pop               

Seperate target variable to prevent data leakage and keep aside a validation set to test model on unseen data

In [ ]:
y = X.pop("is_fraud")
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=69)

Remove unncessary symbols from text data for preprocessing

In [ ]:
symb_remv = r'[_,/\-@#&:\s]'

obj_cols = ['merchant', 'category', 'first', 'last', 'street', 'city', 'state', 'job']

for col in obj_cols:
    X_train[col] = X_train[col].str.replace(symb_remv, '', regex=True).str.strip()
    X_valid[col] = X_valid[col].str.replace(symb_remv, '', regex=True).str.strip()

X_train["merchant"] = X_train["merchant"].str.replace("fraud", "", case = False)
X_valid["merchant"] = X_valid["merchant"].str.replace("fraud", "", case = False)

Label Encoding Gender

In [ ]:
gender = LabelEncoder()

X_train["gender"] = gender.fit_transform(X_train["gender"])
X_valid["gender"] = gender.transform(X_valid["gender"])

Frequency encoding the following columns

In [ ]:
freq_encd_cols = ["merchant", "first", "last", "category", "street", "city", "state", "job"]

for col in freq_encd_cols:

    freq_map = X_train[col].value_counts()

    X_train[col] = X_train[col].map(freq_map)
    X_valid[col] = X_valid[col].map(freq_map)

Checking mutual information scores to pick most useful features

In [ ]:
num_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]

mi_scores = mutual_info_classif(X_train_copy[num_cols], y_train, discrete_features=False)

mi_scores_df = pd.DataFrame({'Feature': num_cols, 'MI_Score': mi_scores})
mi_scores_df = mi_scores_df.sort_values(by='MI_Score', ascending=False)

sns.barplot(x='MI_Score', y='Feature', data = mi_scores_df, palette='viridis')

Checking correlation matrix to see any unseen relations by mutual_info_classif

In [ ]:
X_train[num_cols].corrwith(y_train)
plt.figure(figsize=(10, 6))
sns.heatmap(X_train[num_cols].corr(),annot = True, fmt = '.2f')

Picking the most important columns for training

In [ ]:
X_train1 = pd.DataFrame(X_train[["gender", "category", "amt", "state", "unix_time", "zip", "lat", "long", "cc_num"]])
X_valid1 = pd.DataFrame(X_valid[["gender", "category", "amt", "state", "unix_time", "zip", "lat", "long", "cc_num"]])

Since dataset is imbalanced, using this ratio for better model quality

In [ ]:
ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

Baseline model to compare performance with

In [ ]:
dt_model = DecisionTreeClassifier(random_state = 69)

dt_model.fit(X_train1, y_train)

preds = dt_model.predict(X_valid1)

report = classification_report(y_valid, preds, target_names = ["Not Fraud", "Fraud"])

print("Classification report is:")
print(report)

Round 1
Classification report is:
              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00    257818
       Fraud       0.66      0.67      0.66      1517

    accuracy                           1.00    259335
   macro avg       0.83      0.83      0.83    259335
weighted avg       1.00      1.00      1.00    259335

Round 2
Classification report is:
              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00    257818
       Fraud       0.66      0.67      0.66      1517

    accuracy                           1.00    259335
   macro avg       0.83      0.83      0.83    259335
weighted avg       1.00      1.00      1.00    259335

Round 3
Classification report is:
              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00    257818
       Fraud       0.66      0.67      0.66      1517

    accuracy                           1.00    259335
   macro avg       0.83   

Optuna for best hyperparameter searching

In [ ]:
def objective(trial):
    param = {
        'eval_metric': 'auc',
        'reg_lambda': trial.suggest_float('lambda', 1.0, 4.0),
        'max_depth': trial.suggest_int('max_depth', 8, 16),
        'eta': trial.suggest_float('eta', 0.01, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'tree_method':'hist',
        'n_estimators': 1000,
        'early_stopping_rounds':100,
        'scale_pos_weight': ratio,
        'device': 'cuda',
        'random_state': 69
    }

    model = XGBClassifier(**param)
    model.fit(X_train1, y_train, eval_set = [(X_valid1, y_valid)], verbose = 100)
    preds = model.predict_proba(X_valid1)[:, 1]
    score = roc_auc_score(y_valid, preds)
    return score

xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(objective, n_trials=10)

[I 2024-07-19 10:24:07,960] A new study created in memory with name: no-name-9ea6c649-7400-46be-acc7-9676f9725981


[0]	validation_0-auc:0.99263
[100]	validation_0-auc:0.99221


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:24:11] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-07-19 10:24:11,456] Trial 0 finished with value: 0.9932179587903355 and parameters: {'lambda': 1.8164146196947442, 'max_depth': 11, 'eta': 0.010401855294423855, 'subsample': 0.6034582604697567, 'colsample_bylevel': 0.9591255838468815, 'colsample_bytree': 0.7261032463568482}. Best is trial 0 with value: 0.9932179587903355.


[0]	validation_0-auc:0.95365
[100]	validation_0-auc:0.99705
[200]	validation_0-auc:0.99870
[300]	validation_0-auc:0.99896
[400]	validation_0-auc:0.99914
[500]	validation_0-auc:0.99925
[600]	validation_0-auc:0.99931
[700]	validation_0-auc:0.99937
[800]	validation_0-auc:0.99939
[900]	validation_0-auc:0.99941
[999]	validation_0-auc:0.99941


[I 2024-07-19 10:24:37,835] Trial 1 finished with value: 0.9994152538800692 and parameters: {'lambda': 3.1746001035102163, 'max_depth': 16, 'eta': 0.07628949004137456, 'subsample': 0.5540391318261598, 'colsample_bylevel': 0.8209052774575794, 'colsample_bytree': 0.5064089983052313}. Best is trial 1 with value: 0.9994152538800692.


...

[0]	validation_0-auc:0.98340
[100]	validation_0-auc:0.99467
[200]	validation_0-auc:0.99718
[300]	validation_0-auc:0.99812
[400]	validation_0-auc:0.99857
[500]	validation_0-auc:0.99880
[600]	validation_0-auc:0.99891
[700]	validation_0-auc:0.99900
[800]	validation_0-auc:0.99905
[900]	validation_0-auc:0.99907
[999]	validation_0-auc:0.99912


[I 2024-07-19 10:26:58,371] Trial 8 finished with value: 0.9991207100747788 and parameters: {'lambda': 2.1249026278219527, 'max_depth': 14, 'eta': 0.04791877620726165, 'subsample': 0.5359352188691379, 'colsample_bylevel': 0.5634259065621844, 'colsample_bytree': 0.6620619588737597}. Best is trial 1 with value: 0.9994152538800692.


[0]	validation_0-auc:0.97906
[100]	validation_0-auc:0.99230
[200]	validation_0-auc:0.99621
[300]	validation_0-auc:0.99730
[400]	validation_0-auc:0.99801
[500]	validation_0-auc:0.99845
[600]	validation_0-auc:0.99867
[700]	validation_0-auc:0.99889
[800]	validation_0-auc:0.99897
[900]	validation_0-auc:0.99904
[999]	validation_0-auc:0.99909


[I 2024-07-19 10:27:31,122] Trial 9 finished with value: 0.9990923663794903 and parameters: {'lambda': 3.8377055587891378, 'max_depth': 16, 'eta': 0.03288005704220766, 'subsample': 0.8056445302439523, 'colsample_bylevel': 0.5468460619320108, 'colsample_bytree': 0.6410046673794343}. Best is trial 1 with value: 0.9994152538800692.


NameError: name 'study' is not defined

In [ ]:
xgb_study.best_params

{'lambda': 3.1746001035102163,
 'max_depth': 16,
 'eta': 0.07628949004137456,
 'subsample': 0.5540391318261598,
 'colsample_bylevel': 0.8209052774575794,
 'colsample_bytree': 0.5064089983052313}

Training optimal XgBoost model and checking performance

In [ ]:
xgb_model = XGBClassifier(reg_lambda = 3.1746001035102163, max_depth = 16, eta = 0.07628949004137456, subsample = 0.5540391318261598, colsample_bylevel = 0.8209052774575794,
                      colsample_bytree = 0.5064089983052313, eval_metric = 'auc', n_estimators = 1000, device = 'cuda', scale_pos_weight = ratio,
                      early_stopping_rounds = 100, random_state = 69)

xgb_model.fit(X_train1, y_train, eval_set = [(X_valid1, y_valid)], verbose = False)

preds = xgb_model.predict(X_valid1)

report = classification_report(y_valid, preds, target_names = ["Not Fraud", "Fraud"])

print("Classification report is:")
print(report)

Classification report is:
              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00    257818
       Fraud       0.85      0.90      0.88      1517

    accuracy                           1.00    259335
   macro avg       0.93      0.95      0.94    259335
weighted avg       1.00      1.00      1.00    259335

